In [ ]:
from words import WordData
from nlp import get_stem

from paths import word_path, anki_csv_path, sentence_data_path

In [ ]:
all_words = []

In [ ]:
import csv

with open(anki_csv_path, newline='\n') as f:
    reader = csv.reader(f, delimiter='\t')
    next(reader)

    for line in reader:

        word = line[4].split(',')[0].split(' ')[0]
        subdeck = line[3].split('_')[0]

        word_data = WordData(word)
        word_data.stem = get_stem(word)
        word_data.subdeck = subdeck

        all_words.append(word_data)


In [ ]:
import pickle
with open(sentence_data_path, "rb") as f:
    all_sentences = pickle.load(f)

In [ ]:
from collections import defaultdict
from itertools import cycle

def choose_sentences(sentences, stem, max_samples=3):
    # Group sentences by the actual word corresponding to the stem
    grouped_sentences = defaultdict(list)
    for sentence in sentences:
        word = sentence.stem_mappings[stem]
        grouped_sentences[word].append(sentence)

    # Sort sentences within each group by their length (shortest first)
    for group in grouped_sentences.values():
        group.sort(key=lambda s: len(s.text))

    # Create a round-robin iterator over the sentence groups
    round_robin_iterator = cycle(grouped_sentences.values())

    # Select sentences ensuring diversity in actual words, and choosing the shortest one from each group
    selected_sentences = []
    while len(selected_sentences) < min(max_samples, len(sentences)):
        try:
            group = next(round_robin_iterator)
            if group:  # Check if the group still has sentences left
                selected_sentences.append(group.pop(0))  # Select the shortest sentence
        except StopIteration:
            break  # Break if all groups are exhausted

    return selected_sentences

sentences = list(filter(lambda sentence: get_stem("delen")  in sentence.stem_mappings, all_sentences.values()))
sentences
print(choose_sentences(sentences, get_stem("delen")))

In [ ]:
import random


for word_data in all_words:
    stem = word_data.stem
    sentences = list(filter(lambda sentence: stem in sentence.stem_mappings, all_sentences.values()))
    
    sampled = choose_sentences(sentences, stem)
    
    word_data.sentences_text = list(map(lambda sentence: sentence.text, sampled))
    word_data.sentences_translation = list(map(lambda sentence: sentence.translation, sampled))
    word_data.sentences_audio = list(map(lambda sentence: sentence.audio_id, sampled))

random.choice(all_words)


In [ ]:
with open(word_path, "wb") as f:
    pickle.dump(all_words, f)